# これを300 epochほどやろう

In [6]:
# coding: UTF-8
# これは成功！（100 epochで十分）

import argparse
import os

import numpy as np
#from PIL import Image
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import initializers
from chainer import training
from chainer import Variable
from chainer import cuda
from chainer.dataset import iterator as iterator_module
from chainer.training import extensions
from chainer.dataset import convert
from chainer import Variable
#from evaluate import out_generated_image
from chainer import variable
from data import GmmDataset
from utils import out_generated, calcEMD

# np = cuda.cupy
#random_state = np.random.RandomState(123)


# Updater

class WGANUpdater(training.StandardUpdater):
    def __init__(self, iterator, generator, critic,
                 n_c, opt_g, opt_c, lam, lam2, n_hidden, device):
        if isinstance(iterator, iterator_module.Iterator):
            iterator = {'main': iterator}
        self._iterators = iterator
        self.generator = generator
        self.critic = critic
        self.n_c = n_c
        self._optimizers = {'generator': opt_g, 'critic': opt_c}
        self.lam = lam
        self.lam2 = lam2
        self.device = device
        self.converter = convert.concat_examples
        self.iteration = 0
        self.n_hidden = n_hidden

    def update_core(self):
        batch = self.get_iterator('main').next()
        batchsize = len(batch)

        # Step1 Generate
        z = Variable(np.asarray(self.generator.make_hidden(batchsize)))
        #print(z)
        x_gen = self.generator(z)
        #print("iteration is ", self.iteration)
        #print("x_gen is ", x_gen)
        y_gen = self.critic(x_gen)

        # Step2 real
        x_real = Variable(np.array(batch))
        y_real = self.critic(x_real)

        # Step3 Compute loss for wgan_gp
        eps = np.random.uniform(0, 1, (batchsize, 1)).astype("f")
        x_mid = eps * x_real + (1.0 - eps) * x_gen
        x_mid_v = Variable(x_mid.data)
        y_mid = self.critic(x_mid_v)
        dydx = chainer.grad([y_mid], [x_mid_v], enable_double_backprop=True)[0]
        dydx = F.sqrt(F.sum(F.square(dydx), axis=1))
        loss_gp = self.lam * F.mean_squared_error(dydx, np.ones_like(dydx.data))
        loss_cri = F.sum(-y_real) / batchsize
        #print(loss_cri)
        loss_cri += F.sum(y_gen) / batchsize
        
        loss_sp = self.lam2 * F.absolute(F.sum(self.critic.inter.W) - 1)
        loss_all = loss_cri + loss_gp + loss_sp
        
        # Step4 Update critic
        self.critic.cleargrads()
        loss_all.backward()
        self._optimizers['critic'].update()

        # Step5 Update generator
        
        if self.iteration % self.n_c == 0:
            loss_gen = F.sum(-y_gen) / batchsize
            #print(loss_gen)
            self.generator.cleargrads()
            loss_gen.backward()
            self._optimizers['generator'].update()
            chainer.reporter.report({'loss/generator': loss_gen})

        """
        if self.iteration < 2500 and self.iteration % 100 == 0:
            loss_gen = F.sum(-y_gen) / batchsize
            loss_sp = self.lam2 * F.absolute(F.sum(self.generator.inter.W) - 1)
            loss_gen += loss_sp
            self.generator.cleargrads()
            loss_gen.backward()
            self._optimizers['generator'].update()
            chainer.reporter.report({'loss/generator': loss_gen})

        if self.iteration > 2500 and self.iteration % self.n_c == 0:
            loss_gen = F.sum(-y_gen) / batchsize
            loss_sp = self.lam2 * F.absolute(F.sum(self.generator.inter.W) - 1)
            loss_gen += loss_sp
            self.generator.cleargrads()
            loss_gen.backward()
            self._optimizers['generator'].update()
            chainer.reporter.report({'loss/generator': loss_gen})
        """

        # Step6 Report
        chainer.reporter.report({'loss/critic': loss_cri})


def main():
    parser = argparse.ArgumentParser(description='WGAN')
    parser.add_argument('--batchsize', '-b', type=int, default=64,
                        help='Number of images in each mini-batch')
    parser.add_argument('--datasize', '-d', type=int, default=10000,
                        help='Number of samples')
    parser.add_argument('--lam', '-l', type=int, default=0.5,
                        help='Hyperparameter of gp')
    parser.add_argument('--lam2', '-l2', type=int, default=1.0,
                        help='Hyperparameter of gp')
    parser.add_argument('--n_hidden', type=int, default=100,
                        help='latent variable')
    parser.add_argument('--seed', '-s', type=int, default=111,
                        help='seed number')
    parser.add_argument('--epoch', '-e', type=int, default=300,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--gpu', '-g', type=int, default=-1,
                        help='GPU ID (negative value indicates CPU)')
    parser.add_argument('--out', '-o', default='result',
                        help='Directory to output the result')
    parser.add_argument('--resume', '-r', default='',
                        help='Resume the training from snapshot')
    parser.add_argument('--unit', '-u', type=int, default=1000,
                        help='Number of units')
    parser.add_argument('--setting', '-set', type=int, default=11,
                        help='Number of units')
    args = parser.parse_args(args=[])

    # parameter set
    ## Optimizers
    alpha = 0.002
    beta1 = 0.9
    beta2 = 0.999

    ## Network
    dim = 2
    num_nets = 2
    wscale = 0.02

    # Networks
    generator = Generator(dim, num_nets, args.n_hidden, wscale)
    critic = Critic(num_nets, wscale)

    if args.gpu >= 0:
        chainer.cuda.get_device(args.gpu).use()
        generator.to_gpu()
        critic.to_gpu()

    # Optimizer set
    opt_g = chainer.optimizers.Adam(alpha = alpha, beta1 = beta1, beta2 = beta2)
    #opt_g = chainer.optimizers.RMSprop(lr=0.00005)
    opt_g.setup(generator)
    #opt_g.add_hook(chainer.optimizer.WeightDecay(0.00001), 'hook_dec')
    opt_g.add_hook(chainer.optimizer.GradientClipping(1))

    opt_c = chainer.optimizers.Adam(alpha = alpha, beta1 = beta1, beta2 = beta2)
    #opt_c = chainer.optimizers.RMSprop(lr=0.00005)
    opt_c.setup(critic)
    #opt_c.add_hook(chainer.optimizer.WeightDecay(0.00001), 'hook_dec')
    opt_c.add_hook(chainer.optimizer.GradientClipping(1))

    # Dataset
    """
    train, test = chainer.datasets.get_mnist(withlabel=False, ndim=3, scale=255.)
    train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
    """

    dataset = GmmDataset(args.datasize, args.seed, std=0.02, scale=2)
    train_iter = chainer.iterators.SerialIterator(dataset, args.batchsize)

    # Trainer
    updater = WGANUpdater(train_iter, generator, critic, 5, opt_g, opt_c, args.lam, args.lam2, args.n_hidden, device=args.gpu)
    trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

    # Extensions
    # trainer.extend(extensions.dump_graph('wasserstein distance'))
    """
    epoch_interval = (50, 'epoch')
    trainer.extend(extensions.snapshot(filename='2dim2snap_epoch_{.updater.epoch}.npz'), trigger=epoch_interval)
    trainer.extend(extensions.snapshot_object(generator, '2dim2gensnap_epoch_{.updater.epoch}.npz'), trigger=epoch_interval)
    trainer.extend(extensions.snapshot_object(critic, '2dim2dissnap_epoch_{.updater.epoch}.npz'), trigger=epoch_interval)
    """
    
    trainer.extend(extensions.LogReport())

    trainer.extend(
        extensions.PlotReport(['loss/critic'],
                              'epoch', file_name='criticloss_2dim%d_2.png' % args.setting))

    trainer.extend(
        extensions.PlotReport(
            ['loss/generator'], 'epoch', file_name='genloss_2dim%d_2.png' % args.setting))

    trainer.extend(extensions.PrintReport(
        ['epoch', 'loss/critic', 'loss/generator', 'elapsed_time']))

    out = "/Users/keiikegami/Dropbox/research/Imaizumi-Sensei/paper_result/2dim/result%d_2" % args.setting

    trainer.extend(calcEMD(generator), trigger=(1, 'epoch'))
    trainer.extend(extensions.LogReport(log_name = "log_2dim%d_2" % args.setting))
    trainer.extend(out_generated(generator, args.seed + 10, out, radius=2), trigger=(10, 'epoch'))
    #trainer.extend(extensions.ProgressBar())

    if args.resume:
        chainer.serializers.load_npz(args.resume, trainer)

    # Run
    trainer.run()


# orriginal class of link
class Intersection2(chainer.Link):

    def __init__(self, outdim, numnet):
        super(Intersection2, self).__init__()
        self.outdim = outdim
        self.numnet = numnet
        with self.init_scope():
            W = chainer.initializers.One()
            self.W = variable.Parameter(W)
            self.W.initialize((self.numnet, 1))

    def __call__(self, x):
        if self.outdim == 1:
            weight = F.relu(self.W.T)
        else:
            weight = F.relu(self.make_weight(self.W))

        return F.matmul(weight, x)

    def make_weight(self, array):
        weight_matrix = np.zeros((self.outdim, self.outdim * self.numnet), dtype=np.float32)
        for i in range(self.numnet):
            q = np.array(array[i, 0].data, dtype=np.float32)
            weight_matrix[:, i * self.outdim:(i + 1) * self.outdim] = np.identity(self.outdim, dtype=np.float32) * q
        return Variable(weight_matrix)

    
class Generator(chainer.Chain):

    def __init__(self, dim=784, num_nets=784, latent=100, wscale=0.02):
        super(Generator, self).__init__()
        self.dim = dim
        self.num_nets = num_nets
        self.wscale = wscale
        self.n_hidden = latent

        with self.init_scope():
            self.inter = Intersection2(self.dim, self.num_nets)

            for net in range(self.num_nets):
                w = chainer.initializers.Normal(self.wscale)
                b = chainer.initializers.Normal(self.wscale)

                setattr(self, "l1_{}".format(net), L.Linear(None, 48, initialW=w, initial_bias=b))
                setattr(self, "l2_{}".format(net), L.Linear(None, 48, initialW=w, initial_bias=b))
                setattr(self, "l3_{}".format(net), L.Linear(None, 2, initialW=w, initial_bias=b))

                # set batchnormalization
                #setattr(self, "bn1_{}".format(net), L.BatchNormalization(size=800))

    def make_hidden(self, batchsize):
        return np.random.uniform(-1, 1, (batchsize, self.n_hidden)).astype(np.float32)

    def __call__(self, z, test=False):

        for net in range(self.num_nets):
            #h = F.relu(getattr(self, 'bn1_{}'.format(net))(getattr(self, 'l1_{}'.format(net))(z)))
            h = F.relu(getattr(self, 'l1_{}'.format(net))(z))
            h2 = F.relu(getattr(self, 'l2_{}'.format(net))(h))
            # ここがreluじゃダメなの自明でしょ
            h2 = getattr(self, 'l3_{}'.format(net))(h2)

            if net == 0:
                X = h2
            else:
                X = F.concat((X, h2), axis=1)

        batchsize = X.shape[0]
        X = X.reshape(batchsize, self.num_nets * self.dim)
        x = self.inter(X.T).T
        #x = Variable(np.reshape(x, (batchsize, 1, 28, 28)))
        #x = Variable(x)
        return x


class Critic(chainer.Chain):
    def __init__(self, num_nets=784, wscale=0.02):
        super(Critic, self).__init__()
        self.num_nets = num_nets
        self.wscale = wscale

        with self.init_scope():
            self.inter = Intersection2(1, self.num_nets)

            for net in range(self.num_nets):
                w = chainer.initializers.Normal(self.wscale)
                b = chainer.initializers.Normal(self.wscale)

                setattr(self, "l1_{}".format(net), L.Linear(None, 48, initialW=w, initial_bias=b))
                setattr(self, "l2_{}".format(net), L.Linear(None, 48, initialW=w, initial_bias=b))
                setattr(self, "l3_{}".format(net), L.Linear(None, 1, initialW=w, initial_bias=b))

                # set batchnormalization
                #setattr(self, "bn1_{}".format(net), L.BatchNormalization(size=800))
                #setattr(self, "bn2_{}".format(net), L.BatchNormalization(size=800))

    def __call__(self, x, test=False):

        x = x.reshape(64, 2)
        for net in range(self.num_nets):
            # ここでhがnanになることで全てがnanになる（xは確かにnanではない）（そしてそれはその前のupdateでWがnanになってるから）
            #h = F.leaky_relu(getattr(self, 'bn1_{}'.format(net))(getattr(self, 'l1_{}'.format(net))(x)))
            h = F.leaky_relu(getattr(self, 'l1_{}'.format(net))(x))
            h = F.leaky_relu(getattr(self, 'l2_{}'.format(net))(h))
            # ここsumやめる
            #h2 = F.sum(getattr(self, 'bn2_{}'.format(net))(getattr(self, 'l2_{}'.format(net))(h)), axis=1)
            h2 = getattr(self, 'l3_{}'.format(net))(h)

            if net == 0:
                #Y = h2.reshape(64, 1)
                Y = h2

            else:
                #Y = F.concat((Y, h2.reshape(64, 1)), axis=1)
                Y = F.concat((Y, h2), axis = 1)

        y = self.inter(Y.T)

        return y


In [7]:
if __name__ == "__main__":
    main()

epoch       loss/critic  loss/generator  elapsed_time
1           -2.95842     1.34842         3.35822       
2           -4.06097     4.02864         7.78699       
3           -4.19873     3.64111         12.4905       
4           -4.36702     3.0907          17.7042       
5           -3.89828     3.71337         23.5644       
6           -3.68612     3.99081         29.9001       
7           -2.36234     4.51101         36.517        
8           -2.3077      9.51512         42.2427       
9           -2.28919     12.686          47.9306       
10          -2.04593     13.3311         53.4094       
11          -1.88287     13.1423         59.3895       
12          -1.6314      13.5262         64.8872       
13          -1.43448     13.405          70.3123       
14          -1.22839     14.0939         75.387        
15          -0.961484    14.257          80.651        
16          -0.933731    14.9784         86.0527       
17          -1.00986     15.3274         91.5757  

139         -0.230832    4.55255         626.958       
140         -0.249925    4.57133         631.745       
141         -0.330778    4.54203         636.919       
142         -0.244481    4.68622         641.76        
143         -0.242191    4.39126         646.456       
144         -0.239191    4.51099         651.216       
145         -0.227539    4.50903         656.019       
146         -0.22009     4.30504         660.775       
147         -0.23187     4.29383         665.602       
148         -0.255794    4.32861         670.433       
149         -0.264943    4.4             675.353       
150         -0.238836    4.25652         680.203       
151         -0.315264    4.35599         685.236       
152         -0.231455    4.46855         690.024       
153         -0.241384    4.3923          694.984       
154         -0.236135    4.44435         699.908       
155         -0.21193     4.39474         704.856       
156         -0.218226    4.19191         709.761

278         -0.149173    1.87158         1341.94       
279         -0.151584    1.9003          1347.5        
280         -0.145957    1.8216          1352.96       
281         -0.233264    1.8428          1358.87       
282         -0.149212    1.9317          1364.42       
283         -0.166588    1.8568          1369.81       
284         -0.166682    1.67661         1375.4        
285         -0.1543      1.72801         1381.27       
286         -0.149894    1.85384         1386.93       
287         -0.137245    1.929           1391.94       
288         -0.13835     2.01394         1396.82       
289         -0.151454    2.1266          1401.68       
290         -0.164925    2.07326         1406.76       
291         -0.243792    2.15086         1412.3        
292         -0.154255    2.24664         1417.27       
293         -0.16115     2.37786         1422.36       
294         -0.137791    2.22294         1427.42       
295         -0.128854    2.08234         1432.55